## ライブラリ定義

In [13]:
from __future__ import print_function
from builtins import input
from builtins import range

import pyfftw   # See https://github.com/pyFFTW/pyFFTW/issues/40
import numpy as np
import functools
import operator
import matplotlib.pyplot as mplot
mplot.rcParams["axes.grid"] = False
import math
import pprint
import os
import shutil
import time

from scipy.linalg import toeplitz
from sporco.dictlrn import cbpdndl
from sporco.admm import cbpdn
from sporco import util
from sporco import plot
from sporco import cnvrep
import sporco.linalg as sl
import sporco.metric as sm
from sporco.admm import ccmod
from skimage.measure import compare_ssim
from skimage.measure import compare_psnr
plot.config_notebook_plotting()

In [15]:
def getimages():
    exim = util.ExampleImages(scaled=True, zoom=0.5, gray=True)
    S1 = exim.image('barbara.png', idxexp=np.s_[10:522, 100:612])
    S2 = exim.image('kodim23.png', idxexp=np.s_[:, 60:572])
    S3 = exim.image('monarch.png', idxexp=np.s_[:, 160:672])
    S4 = exim.image('sail.png', idxexp=np.s_[:, 210:722])
    S5 = exim.image('tulips.png', idxexp=np.s_[:, 30:542])
    return np.dstack((S1, S2, S3, S4, S5))

## 学習用画像をダウンロード

In [30]:
S = getimages().astype(np.float32)
print("%d images, each is %dx%d." % (S.shape[2], S.shape[0], S.shape[1]))
print(S[0][0][0])

5 images, each is 256x256.
0.85105884


## 学習用画像の前処理

In [18]:
Sl = np.zeros_like(S)
Smean = np.mean(S*2, axis=(0, 1))
Sh = S*2 - Smean

## 辞書、係数の初期化

In [19]:
#TODO: explicitly zero-padding (for me, foolish)
D = np.random.randn(12, 12, 256)
cri = cnvrep.CSC_ConvRepIndexing(D, S)
Dr0 = np.asarray(D.reshape(cri.shpD), dtype=S.dtype)
Slr = np.asarray(Sl.reshape(cri.shpS), dtype=S.dtype)
Shr = np.asarray(Sh.reshape(cri.shpS), dtype=S.dtype)
#Shf = sl.rfftn(Shr, s=cri.Nv, axes=cri.axisN) # implicitly zero-padding

crop_op = []
for l in Dr0.shape:
    crop_op.append(slice(0, l))
crop_op = tuple(crop_op)
Dr0 = cnvrep.getPcn(Dr0.shape, cri.Nv, cri.dimN, cri.dimCd, zm=False)(cnvrep.zpad(Dr0, cri.Nv))[crop_op]

## テスト用画像

In [29]:
#テスト用画像をダウンロード

# scales=Trueのとき、画像を正規化
# zoom:画像のスケールを調整
exim1 = util.ExampleImages(scaled=True, zoom=0.5, pth='./')
S1_test = exim1.image('couple.tiff') #(256, 256)
#print(S1_test.shape)
exim2 = util.ExampleImages(scaled=True, zoom=1, pth='./')
S2_test = exim2.image('LENNA.bmp') #(256, 256)
#画像2枚を1つにまとめる
S_test = np.dstack((S1_test, S2_test)) #(x,y,枚数)

#畳み込み型辞書学習に必要な変数の次元を参照
cri_test = cnvrep.CSC_ConvRepIndexing(D, S_test)
# shpS(N0,  N1, ...,  C,   K,   1) C is the number of channels in S
# shpD(N0,  N1, ...,  1,   1,   M) K is the number of signals in S
# shpX(N0,  N1, ...,  C,   K,   M) M is the number of filters in D
print("cri_test"+"\n"+str(cri_test))
Sl_test, Sh_test = util.tikhonov_filter(S_test, 5, 16) #Tikhonov正則化
Slr_test = np.asarray(Sl_test.reshape(cri_test.shpS), dtype=S_test.dtype)
Shr_test = np.asarray(Sh_test.reshape(cri_test.shpS), dtype=S_test.dtype)

cri_test
{'C': 1,
 'Cd': 1,
 'K': 2,
 'M': 256,
 'N': 65536,
 'Nv': (256, 256),
 'axisC': 2,
 'axisK': 3,
 'axisM': 4,
 'axisN': (0, 1),
 'dimC': 0,
 'dimCd': 0,
 'dimK': 1,
 'dimN': 2,
 'shpD': (12, 12, 1, 1, 256),
 'shpS': (256, 256, 1, 2, 1),
 'shpX': (256, 256, 1, 2, 256)}
